# Assignment 2

Deadline: 26.03.2025, 12:00 CET

<Add your name, student-id and emal address>

## 1. Linearization of Turnover

**(15 points)**

Turnover constraints are used to limit the amount of change in portfolio weights between periods, helping to manage transaction costs and maintain portfolio stability.

Your task is to implement a method `linearize_turnover_constraint` for the class `QuadraticProgram`, which modifies the quadratic programming problem to incorporate a linearized turnover constraint. This will involve updating the objective function coefficients, equality and inequality constraints, as well as the lower and upper bounds of the problem. 

Additionally, complete the example provided below to demonstrate that your method functions correctly.

In class, we discussed a solution that involved augmenting the dimensionality by a factor of three. Here, you are asked to implement an alternative method that involves a two-fold increase in dimensions. If you are unable to implement the two-fold method, you may proceed with the three-fold approach.

### Function Parameters:
- `x_init` (np.ndarray): The initial portfolio weights.
- `to_budget` (float, optional): The maximum allowable turnover. Defaults to `float('inf')`.

### Steps for Function Implementation:

As discussed in the lecture, introduce auxiliary variables and augment the matrices/vectors used for optimization.

- **Objective Function Coefficients**:  
  Pad the existing objective function coefficients (`P` and `q`) to accommodate the new variables introduced by the turnover constraint.  
  *Note*: "Padding" refers to adding extra elements (typically zeros) to an array or matrix to increase its size to a desired shape.

- **Equality Constraints**:  
  Pad the existing equality constraint matrix (`A`) to account for the new variables.

- **Inequality Constraints**:  
  Pad the existing inequality constraint matrix ('G') and vector ('h') and further add a new inequality constraint row to incorporate the turnover constraint.  

- **Lower and Upper Bounds**:  
  Pad the existing lower (`lb`) and upper (`ub`) bounds to accommodate the new variables.

- **Update Problem Data**:  
  Overwrite the original problem data in the `QuadraticProgram` class with the updated matrices and vectors to include the linearized turnover constraint.

In [2]:
# Import standard libraries
import types
import os
import sys

# Import third-party libraries
import numpy as np
import pandas as pd

# Import local modules
project_root = os.path.dirname(os.path.dirname(os.getcwd()))  # Change this path if needed
src_path = os.path.join(project_root, 'qpmwp-course/src')
sys.path.append(project_root)
sys.path.append(src_path)
from estimation.covariance import Covariance
from estimation.expected_return import ExpectedReturn
from optimization.constraints import Constraints
from optimization.quadratic_program import QuadraticProgram
from helper_functions import load_data_msci

In [ ]:
def linearize_turnover_constraint(self, x_init: np.ndarray, to_budget=float('inf')) -> None:
        '''
        Linearize the turnover constraint in the quadratic programming problem.

        This method modifies the quadratic programming problem to include a linearized turnover constraint.

        Parameters:
        -----------
        x_init : np.ndarray
            The initial portfolio weights.
        to_budget : float, optional
            The maximum allowable turnover. Defaults to float('inf').

        Notes:
        ------
        - The method updates the problem's objective function coefficients, inequality constraints,
        equality constraints, and bounds to account for the turnover constraint.
        - The original problem data is overridden with the updated matrices and vectors.

        Examples:
        ---------
        >>> qp = QuadraticProgram(P, q, G, h, A, b, lb, ub, solver='cvxopt')
        >>> qp.linearize_turnover_constraint(x_init=np.array([0.1, 0.2, 0.3]), to_budget=0.05)
        '''
        # Dimensions
        n = len(self.problem_data.get('q'))
        m = 0 if self.problem_data.get('G') is None else self.problem_data.get('G').shape[0]

        # Update the coefficients of the objective function
        # <add code here>

        # Update the equality constraints
        # <add code here>

        # Update the inequality constraints
        # <add code here>

        # Update lower and upper bounds
        # <add code here>

        # Override the original matrices (notice: b does not change)
        self.update_problem_data({
            'P': P,
            'q': q,
            'G': G,
            'h': h,
            'A': A,
            'lb': lb,
            'ub': ub
        })

        return None

In [ ]:
def linearize_turnover_constraint(self, x_init: np.ndarray, to_budget=float('inf')) -> None:
    '''
    Linearize the turnover constraint in the quadratic programming problem.

    This method modifies the quadratic programming problem to include a linearized turnover constraint.
    
    Parameters:
    -----------
    x_init : np.ndarray
        The initial portfolio weights.
    to_budget : float, optional
        The maximum allowable turnover. Defaults to float('inf').
    
    Notes:
    ------
    - The method updates the problem's objective function coefficients, inequality constraints,
      equality constraints, and bounds to account for the turnover constraint.
    - The original problem data is overridden with the updated matrices and vectors.
    '''
    # Dimensions
    n = len(self.problem_data.get('q'))
    m = 0 if self.problem_data.get('G') is None else self.problem_data.get('G').shape[0]
    
    # Auxiliary variables for absolute turnover (|x - x_init|)
    n_aux = n  # One auxiliary variable per asset
    
    # New variable size (original variables + auxiliary variables)
    new_n = n + n_aux
    
    # Extend P (quadratic term remains unchanged for new variables)
    P = np.block([
        [self.problem_data['P'], np.zeros((n, n_aux))],
        [np.zeros((n_aux, new_n))]
    ])
    
    # Extend q (linear term remains unchanged for new variables)
    q = np.concatenate([self.problem_data['q'], np.zeros(n_aux)])
    
    # Update inequality constraints to model |x - x_init| ≤ aux
    if m > 0:
        G_old = self.problem_data['G']
        h_old = self.problem_data['h']
    else:
        G_old = np.empty((0, n))
        h_old = np.empty(0)
    
    G_aux_pos = np.hstack([np.eye(n), -np.eye(n)])  # x - x_init ≤ aux
    G_aux_neg = np.hstack([-np.eye(n), -np.eye(n)])  # -x + x_init ≤ aux
    G_turnover = np.hstack([np.zeros((1, n)), np.ones((1, n_aux))])  # sum(aux) ≤ to_budget
    
    G = np.vstack([G_old, G_aux_pos, G_aux_neg, G_turnover])
    h = np.concatenate([h_old, x_init, -x_init, [to_budget]])
    
    # Equality constraints remain unchanged
    A = self.problem_data.get('A', None)
    b = self.problem_data.get('b', None)
    
    # Update bounds (auxiliary variables must be non-negative)
    lb = np.concatenate([self.problem_data['lb'], np.zeros(n_aux)])
    ub = np.concatenate([self.problem_data['ub'], np.full(n_aux, np.inf)])
    
    # Override the original matrices (notice: b does not change)
    self.update_problem_data({
        'P': P,
        'q': q,
        'G': G,
        'h': h,
        'A': A,
        'lb': lb,
        'ub': ub
    })

    return None

In [133]:
def linearize_turnover_constraint(self, x_init: np.ndarray, to_budget=float('inf')) -> None:
    """
    Linearize the turnover constraint in the quadratic programming problem.
    """
    n = len(self.problem_data['q'])  # Number of original decision variables
    m = 0 if self.problem_data.get('G') is None else self.problem_data['G'].shape[0]

    # Number of new auxiliary variables
    n_aux = n  

    # Total number of variables after expansion
    total_vars = n + n_aux

    # Expand P (quadratic cost matrix) to accommodate auxiliary variables
    P = np.block([
        [self.problem_data['P'], np.zeros((n, n_aux))],
        [np.zeros((n_aux, n)), np.zeros((n_aux, n_aux))]
    ])

    # Expand q (linear term) to match variable expansion
    q = np.concatenate([self.problem_data['q'], np.zeros(n_aux)])

    # Expand G and h for turnover constraints
    G_old = self.problem_data['G'] if self.problem_data['G'] is not None else np.zeros((0, n))
    h_old = self.problem_data['h'] if self.problem_data['h'] is not None else np.zeros(0)

    # Ensure G_old has the correct shape
    if G_old.shape[0] > 0:
        G_old = np.hstack([G_old, np.zeros((G_old.shape[0], n_aux))])  # Expand columns

    # Constraints for turnover: |x_i - x_init_i| <= u_i
    G_aux_pos = np.hstack([np.eye(n), -np.eye(n_aux)])  # x - u <= x_init
    G_aux_neg = np.hstack([-np.eye(n), -np.eye(n_aux)])  # -x - u <= -x_init

    # Turnover budget constraint: sum(u) <= to_budget
    G_turnover = np.hstack([np.zeros((1, n)), np.ones((1, n_aux))])

    # **Fix: Ensure the expanded G matches the new variable count**
    G = np.vstack([
        G_old,        # (m, n + n_aux)
        G_aux_pos,    # (n, n + n_aux)
        G_aux_neg,    # (n, n + n_aux)
        G_turnover    # (1, n + n_aux)
    ])
    
    h = np.concatenate([
        h_old,       # Original constraints
        x_init,      # x - u <= x_init
        -x_init,     # -x - u <= -x_init
        [to_budget]  # sum(u) <= to_budget
    ])

    # Expand lower and upper bounds
    lb = np.concatenate([self.problem_data['lb'], np.zeros(n_aux)])  # lb for u = 0
    ub = np.concatenate([self.problem_data['ub'], np.ones(n_aux)])  # ub for u = 1

    # **Debugging: Print Final Dimensions**
    print(f"Final G shape: {G.shape}")  # Should be (20 + 2*10 + 1, 20) = (41, 20)
    print(f"Final h shape: {h.shape}")  # Should be (41,)
    print(f"Final lb shape: {lb.shape}")  # Should be (20,)
    print(f"Final ub shape: {ub.shape}")  # Should be (20,)

    # Override problem data
    self.update_problem_data({
        'P': P,
        'q': q,
        'G': G,
        'h': h,
        'lb': lb,
        'ub': ub
    })
    return None

In [134]:
x_init = pd.Series([1/X.shape[1]]*X.shape[1], index=X.columns)
qp.linearize_turnover_constraint(x_init=x_init, to_budget=0.5)
qp.solve()
solution = qp.results.get('solution')
ids = X.columns  # Use the columns of X directly
weights = pd.Series(solution.x[:len(ids)], index=ids)
print("Turnover:")
print(np.abs(weights - x_init).sum())

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 10 and the array at index 1 has size 50

## Demo

#### Create P and q

In [130]:
# Load the msci country index data
N = 10
data = load_data_msci(path = '../data/', n = N)
X = data['return_series']

# Compute the vector of expected returns (mean returns)
q = ExpectedReturn(method='geometric').estimate(X=X, inplace=False)

# Compute the covariance matrix
P = Covariance(method='pearson').estimate(X=X, inplace=False)

# q, P

### Create some constraints, instantiate an object of class QuadraticProgram, and add the method linearize_turnover_constraint to the instance.

In [131]:
# Instantiate the constraints with only the budget and long-only constraints
constraints = Constraints(ids = X.columns.tolist())
constraints.add_budget(rhs=1, sense='=')
constraints.add_box(lower=0.0, upper=1.0)
GhAb = constraints.to_GhAb()

print(GhAb)

# Create a quadratic program and linearize the turnover constraint
qp = QuadraticProgram(
    P = P.to_numpy(),
    q = q.to_numpy() * 0,
    G = GhAb['G'],
    h = GhAb['h'],
    A = GhAb['A'],
    b = GhAb['b'],
    lb = constraints.box['lower'].to_numpy(),
    ub = constraints.box['upper'].to_numpy(),
    solver = 'cvxopt',
)

# Add the linearized turnover constraint method to the instance of class QuadraticProgram
qp.linearize_turnover_constraint = types.MethodType(linearize_turnover_constraint, qp)


{'G': None, 'h': None, 'A': array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]), 'b': array(1.)}


### Add a turnover limit of 50%. Solve the problem and check whether the turnover constraint is respected.

In [135]:
# Prepare initial weights
x_init = pd.Series([1/X.shape[1]]*X.shape[1], index=X.columns)

print(f"Before turnover constraint:")
# print(f"Dimensions of G: {qp.problem_data['G'].shape}")
print(f"Dimensions of lb: {qp.problem_data['lb'].shape}")
print(f"Dimensions of ub: {qp.problem_data['ub'].shape}")
# Add the linearized turnover constraint
qp.linearize_turnover_constraint(x_init=x_init, to_budget=0.5)

print(f"Number of original variables: {X.shape[1]}")
print(f"Dimensions of G: {qp.problem_data['G'].shape}")
print(f"Dimensions of h: {qp.problem_data['h'].shape}")
print(f"Dimensions of lb: {qp.problem_data['lb'].shape}")
print(f"Dimensions of ub: {qp.problem_data['ub'].shape}")

# Solve the problem
qp.solve()

# Check the turnover
solution = qp.results.get('solution')
ids = constraints.ids
weights = pd.Series(solution.x[:len(ids)], index=ids)

print("Turnover:")
print(np.abs(weights - x_init).sum())

Before turnover constraint:
Dimensions of lb: (50,)
Dimensions of ub: (50,)
Number of original variables: 10
Dimensions of G: (20, 10)
Dimensions of h: (20,)
Dimensions of lb: (70,)
Dimensions of ub: (70,)


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 10 and the array at index 1 has size 70